In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep = dataset[['User ID', 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset['Purchased']

In [9]:
indep.shape

(400, 4)

In [10]:
indep

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,True
1,15810944,35,20000,True
2,15668575,26,43000,False
3,15603246,27,57000,False
4,15804002,19,76000,True
...,...,...,...,...
395,15691863,46,41000,False
396,15706071,51,23000,True
397,15654296,50,20000,False
398,15755018,36,33000,True


In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
from sklearn.naive_bayes import CategoricalNB
from sklearn. model_selection import GridSearchCV
param_grid = {'alpha':[1.0], 'force_alpha':[True], 'fit_prior':[True], 'class_prior':[None], 'min_categories':[None]} 
grid = GridSearchCV(CategoricalNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(indep, dep)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Anaconda 3\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [1.0], 'class_prior': [None],
                         'fit_prior': [True], 'force_alpha': [True],
                         'min_categories': [None]},
             scoring='f1_weighted', verbose=3)

In [13]:
result=grid.cv_results_
#print(result)

Y_pred = grid.predict(indep)
#print(y_pred)
   
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(dep, Y_pred)
#print(cm)

from sklearn.metrics import classification_report

clf_report = classification_report(dep, Y_pred)

#print(clf_report)

In [14]:
from sklearn.metrics import f1_score

f1_macro=f1_score(dep,Y_pred,average='weighted')

print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 1.0, 'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'min_categories': None}: 0.9493761140819964


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[254   3]
 [ 17 126]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.99      0.96       257
           1       0.98      0.88      0.93       143

    accuracy                           0.95       400
   macro avg       0.96      0.93      0.94       400
weighted avg       0.95      0.95      0.95       400



In [17]:
#AUC ROC stands for “Area Under the Curve” of the “Receiver Operating Characteristic” curve. 
from sklearn.metrics import roc_auc_score

roc_auc_score(dep,grid.predict_proba(indep)[:,1])

0.9929933879350222

In [18]:
table=pd.DataFrame.from_dict(result)

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,param_min_categories,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,12.021587,3.460937,10.913551,5.952495,1.0,None,True,True,None,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.690376,NaN,0.780251,0.74794,0.773146,NaN,NaN,1
